In [156]:
def importlist(inputtext):
    list_of_lists = []

    with open(inputtext) as f:
        for line in f:
            inner_list = line.split("\n")
            doubleInner = [liner.strip() for liner in inner_list[0].split(" ")]
            if doubleInner[-1] == '':
                doubleInner.pop(-1)
            list_of_lists.append(doubleInner)
    flag = False
    holder = []
    objectives = []
    attributes = []
    for part in list_of_lists:
        part = list(map(int,part))
        if part[0] < 0:
            # it is an obj value
            objectives.append(part)
            flag = True
            continue
        if flag:
            # it is constraints
            attributes.append(part)
            continue
        #it is the first guys
        holder.append(part[0])
    itemCount = holder[0]
    b = holder[1:]
    return itemCount, b, objectives, attributes

In [157]:
importlist("input.txt")

(375, [10100, 10527], 2, 2)